In [ ]:
#Importing all relevant packages 
import gym
from IPython.display import clear_output
from time import sleep
import numpy as np
import random
from IPython.display import clear_output

#Using the Taxi-v3 game in gym and storing in variable env
env = gym.make("Taxi-v3").env

#Providing a render or state instance, resetting again and giving another state instance
#We use this to show the use of reset, render functions and step sample 
env.render()
env.reset() # reset environment to a new, random state
env.render()

#To show the Action and the State Spaces
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [ ]:
#We start by providing a specific state. In our case it is state 428 and we fix this for further problem
#The position in yellow shows the current state of the vehicle (428) and we aim to reach Destination R first which
#is our first goal 
state = env.encode(4, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 428
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [ ]:
#Here we see the possible actions at state 428. 0 is south, 1 is north, 2 is east, 3 is west, 4 is pickup and 5 is dropoff
#We see the possible states the agent can move to and corresponding rewards (-1 for direction movement and -10 for pickup/dropoff)
#We see the rewards issued for each action below
 
env.P[428]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 328, -1, False)],
 2: [(1.0, 448, -1, False)],
 3: [(1.0, 428, -1, False)],
 4: [(1.0, 428, -10, False)],
 5: [(1.0, 428, -10, False)]}

In [148]:
#CASE 1 - WITH RANDOM POLICY (NO REINFORCEMENT LEARNING APPLIED)
#EVALUATION OF ACTIONS
# We clearly see that the timesteps taken and penalties incurred are extremely high
# This is not the right policy for this problem 
env.s = 428  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 841
Penalties incurred: 245


In [149]:
#CASE 1 - WITH RANDOM POLICY (NO REINFORCEMENT LEARNING APPLIED)
#Illustration showing the state shift of the agent for this random policy
#Instead of showing a video capture as per the lectures, we show frames of each state shift back to back

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.05)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 841
State: 0
Action: 5
Reward: 20


In [150]:
#CASE 2 - WITH DEFINED POLICY (Q-LEARNING APPLIED)
#EVALUATION OF ACTIONS
#Resetting the Q-table to all zeros. We start Q-learning policy here onwards
q_table = np.zeros([env.observation_space.n, env.action_space.n])

#Training the agent
#Storing Hyperparameters values. We do not tune them here since we focus on the Policy Definition Aspect.
#We ensure that we apply a Policy that gives us a better approach than the random policy 
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

#Exploration vs Exploitation
#Choosing randon action vs choosing action based on already learnt Q-values
# Iterations kept to 10,000. We can vary this. 
for i in range(1, 10001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value
        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 10000
Training finished.



In [151]:
#Revised Q values for state 428. We will use the highest reward action according to this to reach the next step 
q_table[428]

array([-2.37215359, -2.36390142, -2.37082892, -2.37689387, -4.48600264,
       -6.15881733])

In [152]:
#Evaluation of agent's performance after Q-learning
#We Clearly see that average timesteps per episode has drastically reduced and the penalty is 0
#Much better Evaluation Results after we apply a well defined policy  

total_epochs, total_penalties = 0, 0
episodes = 10

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 10 episodes:
Average timesteps per episode: 12.3
Average penalties per episode: 0.0
